<a href="https://colab.research.google.com/github/jackrankin/chess_engine/blob/main/rc_complex_particletest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
from copy import deepcopy

class robot_coffee:
  def __init__(self):

    self.incr = 1 # tolerance for dx
    self.person = 0
    self.pot0 = 3
    self.pot1 = 10
    self.cup = 7

  # True state
  def step(self, x, a):

    dx0, dx1, dx2 = 0, 0, 0

    # move the robot
    if a == 0 and x[0] > 0:
      dx0 = -self.incr
    elif a == 1 and x[0] < 10:
      dx0 = self.incr

    # pick up the cup
    if a == 2 and x[0] == self.cup:
      dx1 = 1
    else:
      dx1 = x[1]

    # pick up the coffee
    if a == 2 and x[0] == x[3] and x[1]:
      dx2 = 1
    else:
      dx2 = x[2]

    if a == 3:
      dx2 = 0

    return x[0] + dx0, dx1, dx2, x[3]

  def measure(self, x, distribution):
    if distribution[0]: # we expect that the coffee is at 3
      return x[0], x[1], x[2], 3
    return x[0], x[1], x[2], 10

  def cost(self, y):
    if y[2]: return y[0]
    else: return y[3]

  def render(self, x):
    output = ['_']*11
    output[0] = 'P'
    output[3] = 'p'
    output[10] = 'p'

    output[x[3]] = 'P'

    if not x[1]:
      output[7] = 'C'

    output[x[0]] = 'R'

    print(" ".join(output))

    if x[1]:
      print("HAS CUP")
    else:
      print("NO CUP")

    if x[2]:
      print("HAS COFFEE")
    else:
      print("NO COFFEE")

In [48]:
class Particle:
  def __init__(self, distribution, x, sweep_depth, env):
    self.distribution = distribution
    self.env = env
    self.x = x # most cases this is just stored in the env, but since this is a bad env its stored here
    self.sweep_depth = sweep_depth
    self.history = []

  def render(self):
    print(self.env.render(self.x), self.distribution)

  def push_forward(self):
    cost, action, path = self.sweep(self.env.measure(self.x, self.distribution), 0)
    self.x = env.step(self.x, action)

    # if we reached the coffee, update the distribution
    if self.x[0] == self.env.pot0 or self.x[0] == self.env.pot1:
      if self.x[3] == 3:
        self.distribution[0] = 1
        self.distribution[1] = 0
      if self.x[3] == 10:
        self.distribution[0] = 0
        self.distribution[1] = 1

    return self.x, cost, path[::-1]

  def sweep(self, measured_state, depth):
    if depth == self.sweep_depth:
      return (self.env.cost(measured_state), 1, [])

    minimum_cost = float('inf')
    best_action = 0
    best_path = []

    for move in range(4):
      new_state = self.env.step(measured_state, move)
      future_cost, action, path = self.sweep(new_state, depth + 1)

      if future_cost < minimum_cost:
        minimum_cost = future_cost
        best_action = move
        path.append(move)
        best_path = path

    return minimum_cost, best_action, best_path


In [56]:
env = robot_coffee()
x = [np.random.choice(np.arange(0, 11)), 0, 0, np.random.choice([3, 10])] # robot, cup, coffee in cup, coffee position
k = np.random.choice([0,1])
distribution = [k, 1 - k]

sweep_depth = 10

p = Particle(distribution, x, sweep_depth, env)

while p.env.cost(p.env.measure(p.x, p.distribution)):
  p.push_forward()
  p.render()




P _ _ p _ _ _ R _ _ P
HAS CUP
NO COFFEE
None [1, 0]
P _ _ p _ _ R _ _ _ P
HAS CUP
NO COFFEE
None [1, 0]
P _ _ p _ R _ _ _ _ P
HAS CUP
NO COFFEE
None [1, 0]
P _ _ p R _ _ _ _ _ P
HAS CUP
NO COFFEE
None [1, 0]
P _ _ R _ _ _ _ _ _ P
HAS CUP
NO COFFEE
None [0, 1]
P _ _ p R _ _ _ _ _ P
HAS CUP
NO COFFEE
None [0, 1]
P _ _ p _ R _ _ _ _ P
HAS CUP
NO COFFEE
None [0, 1]
P _ _ p _ _ R _ _ _ P
HAS CUP
NO COFFEE
None [0, 1]
P _ _ p _ _ _ R _ _ P
HAS CUP
NO COFFEE
None [0, 1]
P _ _ p _ _ _ _ R _ P
HAS CUP
NO COFFEE
None [0, 1]
P _ _ p _ _ _ _ _ R P
HAS CUP
NO COFFEE
None [0, 1]
P _ _ p _ _ _ _ _ _ R
HAS CUP
NO COFFEE
None [0, 1]
P _ _ p _ _ _ _ _ _ R
HAS CUP
HAS COFFEE
None [0, 1]
P _ _ p _ _ _ _ _ R P
HAS CUP
HAS COFFEE
None [0, 1]
P _ _ p _ _ _ _ R _ P
HAS CUP
HAS COFFEE
None [0, 1]
P _ _ p _ _ _ R _ _ P
HAS CUP
HAS COFFEE
None [0, 1]
P _ _ p _ _ R _ _ _ P
HAS CUP
HAS COFFEE
None [0, 1]
P _ _ p _ R _ _ _ _ P
HAS CUP
HAS COFFEE
None [0, 1]
P _ _ p R _ _ _ _ _ P
HAS CUP
HAS COFFEE
None [0, 1]
P _ _